# Часть I. Нахождение корней. Newton's iteration.

Напишите алгоритм, который создает итерации Ньютона для заданной функции $f(x)$ с известной производной $f'(x)$. Ваш алгоритм должен находить корни $f(x)$ с заданной точностью $\epsilon$.

In [ ]:
def newton_iteration(f, fder, x0, eps=1e-5, maxiter=1000):
    """Нахождение корней $f(x) = 0$ через итерации Ньютона, начиная с x0.
    
    Параметры
    ----------
    f : callable
        Функция, корни которой мы хотим найти.
    fder : callable
        Производная `f`.
    x0 : float
        Начальное приближение итераций Ньютона.
    eps : float
        Заданная точность. 
        Алгоритм прекращает работу когда расстояние между последовательными приближениями меньше `eps`.
        По умолчанию 1e-5.
    maxiter : int
        Максимальное число итераций (по умолчанию 1000.)
        Алгоритм прекращается, когда число итераций достигает `maxiter`.
        Этот параметр нужен лишь для предотвращения бесконечного зацикливания.
    -------
    x : float
        Оценка корня.
    niter : int
        Количество итерацй.
    """
    # ... ENTER YOUR CODE HERE ...

### Тест I.1 

Протестируйте вашу функцию на простом примере  $f(x) = x^2 - 1$ (или похожем). (20% итоговой оценки).

In [ ]:
# ... ENTER YOUR CODE HERE ...

### Тест I.2

Теперь рассмотрим функцию, имеющую несколько корней. Возьмем $f(x) = (x^2 - 1)^2$ в качестве примера. Применим модифицированный алгоритм итераций Ньютона:

$$
x_{n+1} = x_{n} - m \frac{f(x_n)}{f'(x_n)}
$$

и проварьируем $m= 1, 2, 3, 4, 5$. Проверьте количество итераций необходимых для сходимости в пределах фиксированного $\epsilon$. Cогласуются ли ваши наблюдения с ожиданием того, что сходимость будет квадратичной, если $m$ совпадает с количеством корней и линейной в других случаях. (40% итоговой оценки).

In [ ]:
# ... ENTER YOUR CODE HERE ...

# Часть II. Итерации с фиксированной точкой.

Рассмотрим следующее уравнение:

$$
\sqrt{x} = \cos{x}
$$

Постройте левую и правую часть уравнения и оцените решение графически. Оцените локализацию корня "на глаз".

Напишите функцию, находящую решение, используя итерации с фиксированной точкой в пределах погрешности $\epsilon$. Сравните результат с визуальной оценкой.

Далее перепишите задачу фиксированной точки в форме

$$
x = x - \alpha f(x)
$$

где $\alpha$ свободный параметр. Проверьте зависимость количества итераций необходимых для заданой $\epsilon$ от $\alpha$. Сравните результат с ожиданием того, что оптимальное значение $\alpha$ есть

$$
\alpha = \frac{2}{m + M}
$$

где $0 < m < |f'(x)| < M$ вне интервала локлизации. (40% итоговой оценки)

In [ ]:
# ... ENTER YOUR CODE HERE ...

# Part III. Фракталы Ньютона.

(Без оценки). 

Рассмотрим уравнение

$$
x^3 = 1
$$

Оно имеет три решения на комплексной плоскости, $x_k = \exp(\frac{2\pi i\,k}{3})$, $k = 0, 1, 2$.

Итерации Ньютона сходятся к одному из них в зависимости от начального приближения на комплексной плоскости (для сходимости, естественно, нужно задавать комплекснозначную начальную точку.

Постройте области сходимости к различным корням на комплексной плоскости $x$ (В сущности на плоскости $\mathrm{Re}x$ -- $\mathrm{Im}x$). Выполните ряд вычислений, варьируя начальные приближения на сетке значений. Раскрасьте плоскость в три цвета в соответствии с корнем, к которому сходится решение.